In [4]:
import pygame
import sys
from pygame.locals import *

%run logic.ipynb

try:
    pygame.init()
    pygame.font.init()
except pygame.error as e:
    print(f"Pygame initialization error: {e}")
    sys.exit(1)

class SOSGameUI:
    def __init__(self, game):
        self.game = game
        self.screen_width = 700
        self.screen_height = 700
        self.grid_size = 400
        self.start_x = 250
        self.start_y = 100
        
        try:
            self.screen = pygame.display.set_mode((self.screen_width, self.screen_height))
            pygame.display.set_caption("SOS Game")
        except pygame.error as e:
            print(f"Pygame display error: {e}")
            sys.exit(1)
        
        self.BLACK = (0, 0, 0)
        self.WHITE = (255, 255, 255)
        self.GRAY = (200, 200, 200)
        self.BLUE = (0, 0, 255)
        self.RED = (255, 0, 0)
        
        self.running = True
        try:
            self.font = pygame.font.SysFont("Arial", 24)
        except pygame.error:
            self.font = pygame.font.Font(None, 24)  # Use default font if Arial is unavailable
        
        self.input_box = pygame.Rect(30, 230, 100, 40)
        self.input_text = ""
        self.input_active = False  # Track if the input box is active
        self.buttons = []

        self.add_button(30, 70, 160, 40, "Simple", self.set_simple_mode)
        self.add_button(30, 120, 160, 40, "General", self.set_general_mode)
        self.add_button(140, 230, 80, 40, "Refresh", self.update_board_size)
        self.add_button(30, 390, 160, 40, "Swap S/O", self.swap_roles)

    def draw_grid(self):
        self.cell_size = self.grid_size // self.game.n
        for row in range(1, self.game.n):
            pygame.draw.line(self.screen, self.BLACK, (self.start_x + row * self.cell_size, self.start_y),
                             (self.start_x + row * self.cell_size, self.start_y + self.grid_size), 2)
            pygame.draw.line(self.screen, self.BLACK, (self.start_x, self.start_y + row * self.cell_size),
                             (self.start_x + self.grid_size, self.start_y + row * self.cell_size), 2)
        pygame.draw.rect(self.screen, self.BLACK, (self.start_x, self.start_y, self.grid_size, self.grid_size), 2)
    
    def draw_sidebar(self):
        pygame.draw.rect(self.screen, self.GRAY, (10, 10, 220, 680))
        self.draw_text(f"SOS Game Mode: {self.game.mode}", 30, 30, self.BLACK)  # Display current game mode
        self.draw_text("Board Size", 30, 180, self.BLACK)
        
        pygame.draw.rect(self.screen, self.WHITE, self.input_box)
        pygame.draw.rect(self.screen, self.BLACK, self.input_box, 2)
        text_surface = self.font.render(self.input_text, True, self.BLACK)
        self.screen.blit(text_surface, (self.input_box.x + 5, self.input_box.y + 5))
        
        self.draw_text("Swap Roles", 30, 350, self.BLACK)
        
        for rect, text, _ in self.buttons:
            pygame.draw.rect(self.screen, self.WHITE, rect)
            pygame.draw.rect(self.screen, self.BLACK, rect, 2)
            label = self.font.render(text, True, self.BLACK)
            self.screen.blit(label, (rect.x + (rect.width - label.get_width()) // 2, rect.y + (rect.height - label.get_height()) // 2))

    def add_button(self, x, y, width, height, text, action):
        rect = pygame.Rect(x, y, width, height)
        self.buttons.append((rect, text, action))

    def draw_board(self):
        for row in range(self.game.n):
            for col in range(self.game.n):
                cell = self.game.board[row][col]
                if cell:
                    symbol, player = cell
                    color = self.BLUE if player == 'Blue' else self.RED
                    text = self.font.render(symbol, True, color)
                    x = self.start_x + col * self.cell_size + (self.cell_size - text.get_width()) // 2
                    y = self.start_y + row * self.cell_size + (self.cell_size - text.get_height()) // 2
                    self.screen.blit(text, (x, y))
    
    def draw_text(self, text, x, y, color):
        label = self.font.render(text, True, color)
        self.screen.blit(label, (x, y))
    
    def update_board_size(self):
        if self.game.set_board_size(self.input_text):
            self.game.reset_game()
            self.refresh_ui()
    
    def refresh_ui(self):
        self.cell_size = self.grid_size // self.game.n
        self.screen.fill(self.WHITE)
        self.draw_sidebar()
        self.draw_grid()
        self.draw_board()
        pygame.display.flip()
    
    def set_simple_mode(self):
        self.game.set_game_mode("Simple")
        self.game.reset_game()
        self.refresh_ui()
    
    def set_general_mode(self):
        self.game.set_game_mode("General")
        self.game.reset_game()
        self.refresh_ui()
    
    def swap_roles(self):
        self.game.swap_roles()
        self.game.reset_game()
        self.refresh_ui()
    
    def handle_mouse_click(self, pos):
        if self.input_box.collidepoint(pos):
            self.input_active = True
        else:
            self.input_active = False
        for rect, _, action in self.buttons:
            if rect.collidepoint(pos):
                print(f"Button clicked: {_}")  # Debugging print
                action()
                return
        
        if (self.start_x <= pos[0] <= self.start_x + self.grid_size and
            self.start_y <= pos[1] <= self.start_y + self.grid_size):
            col = (pos[0] - self.start_x) // self.cell_size
            row = (pos[1] - self.start_y) // self.cell_size
            self.game.place_symbol(row, col)
            self.refresh_ui()
    
    def run(self):
        while self.running:
            self.screen.fill(self.WHITE)
            self.draw_sidebar()
            self.draw_grid()
            self.draw_board()
            
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.running = False
                elif event.type == pygame.MOUSEBUTTONDOWN:
                    self.handle_mouse_click(event.pos)
                elif event.type == pygame.KEYDOWN:
                    if self.input_active:
                        if event.key == pygame.K_RETURN:
                            self.update_board_size()
                        elif event.key == pygame.K_BACKSPACE:
                            self.input_text = self.input_text[:-1]
                        elif event.unicode.isdigit():
                            self.input_text += event.unicode
            
            pygame.display.flip()
        pygame.quit()
        sys.exit()

if __name__ == "__main__":
    game = SOSGame()
    ui = SOSGameUI(game)
    ui.run()

pygame 2.6.1 (SDL 2.28.4, Python 3.11.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

/opt/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
